In [3]:
import autograd
import numpy as np 
import scipy.stats as st
from autograd import numpy as np_grad
from autograd.scipy import stats as st_grad
import numpy as np

Think about evaluation Greeks for derivatives:

If the pricing is analytical, the situation is straigt-forward: differentiate the function of a price and implement the partial derivative into code. With the use of numerical methods, one can even skip the manual differentiation part

For Monte-Carlo pricing, everything is more complicated. The first idea that may come to mind the the following algoritm:
1) Calculate Monte-Carlo price of a derivative
2) Shift one risk factor by some small $h$
3) Calculate new Monte-Carlo price
4) Use numerical differentiation technique to determine the slope of a function

Problem: both PVs generated by Monte-Carlo are essentially random variables. They get closer to the true value, but are still stochastic. Therefore, the slope between them would have large variance (far larger than two PVs by themselves). Actually, variance of the partial derivative evaluated this way will be:

$$
Var(\Delta) = Var\left(\frac{\hat{C}_n(\theta_0 + h) - \hat{C}_n(\theta_0)}{\bar{h}}\right) = \frac{1}{\bar{h}^2} Var(\hat{C}_n(\theta_0 + h) - \hat{C}_n(\theta_0)) = \frac{1}{\bar{h}^2} (Var(\hat{C}_n(\theta_0 + h)) + Var(\hat{C}_n(\theta_0))) = \frac{2\sigma^2}{n\bar{h}^2} $$

Therefore, $Var(\Delta)$ depends:

* Positively on $\sigma$
* Negatively on $n$ (more simulations = higher precision, just like in regular Monte-Carlo)
* Negatively on $\bar{h}$ (contradicts traditional calculus, in which lower shift positively affects the accuracy of the derivative estimation)

However, higher $h$ increases bias of the simulated sensetivity $\Rightarrow$ Bias-Variance Tradeoff

=======================================================================================================================

However, there is another way: it comes from Machine Learning and by essense is similar to the back-propogation: using chain rule to evaluate the derivative of the output to one of the inputs. It is called **Automatic Adjoint Differentiation**.

In [2]:
# before we move to the full scale Monte-Carlo approach, lets start with something simple: evaluating Greeks without explicitly defining them

class call_option:

    def __init__(self, spot, strike, T, vol, r):
        self.params =  np.array([spot, strike, T, vol, r])

    def price(self, params):
        if params.all() == params.all():
            d1 = (np_grad.log(params[0] / params[1]) + (params[4] + params[3] ** 2 / 2) * params[2]) / (np_grad.sqrt(params[2]) * params[3])
            d2 = d1 - np_grad.sqrt(params[2]) * params[3]
            return params[0] * st_grad.norm.cdf(d1) - params[1] * np_grad.exp(-params[4] * params[2]) * st_grad.norm.cdf(d2)
        else:
            d1 = (np_grad.log(self.params[0] / self.params[1]) + (self.params[4] + self.params[3] ** 2 / 2) * self.params[2]) / (np_grad.sqrt(self.params[2]) * self.params[3])
            d2 = d1 - np_grad.sqrt(self.params[2]) * self.params[3]
            return self.params[0] * st_grad.norm.cdf(d1) - self.params[1] * np_grad.exp(-self.params[4] * self.params[2]) * st_grad.norm.cdf(d2)
    
    def delta(self):
        d1 = (np.log(self.params[0] / self.params[1]) + (self.params[4] + self.params[3] ** 2 / 2) * self.params[2]) / (np.sqrt(self.params[2]) * self.params[3])
        return st.norm.cdf(d1)
    
    def aad_first_order_greeks(self):
        nabla_f = autograd.grad(self.price)
        greeks = nabla_f(self.params)
        # print(greeks)
        print(f"Delta of an option is equal to: {np.round(greeks[0], 4)}")
        print(f"Theta of an option is equal to: {-np.round(greeks[2] / 252, 4)}")
        print(f"Vega of an option is equal to: {np.round(greeks[3], 4)}")
        print(f"Rho of an option is equal to: {np.round(greeks[4], 4)}")
    
    def aad_second_order_greeks(self):
        hess_f = autograd.hessian(self.price)
        greeks = hess_f(self.params)
        # print(greeks)
        print(f"Gamma of an option is equal to: {np.round(greeks[0][0], 4)}")
        print(f"Vanna of an option is equal to: {np.round(greeks[0][3], 4)}")
        print(f"Volga of an option is equal to: {np.round(greeks[3][3], 4)}")
        print(f"Charn of an option is equal to: {-np.round(greeks[0][2], 4)}")

call_option(100.0, 110.0, 1.0, 0.2, 0.05).aad_first_order_greeks()

Delta of an option is equal to: 0.4496
Theta of an option is equal to: -0.0234
Vega of an option is equal to: 39.576
Rho of an option is equal to: 38.9247


In [3]:
# before we move to the full scale Monte-Carlo approach, lets start with something simple: evaluating Greeks without explicitly defining them

class call_option:

    def __init__(self, spot, strike, T, vol, r, N = 0):
        self.params =  np.array([spot, strike, T, vol, r])
        self.N = N

    def price(self, params = []):
        if len(params) != 0:
            d1 = (np_grad.log(params[0] / params[1]) + (params[4] + params[3] ** 2 / 2) * params[2]) / (np_grad.sqrt(params[2]) * params[3])
            d2 = d1 - np_grad.sqrt(params[2]) * params[3]
            return params[0] * st_grad.norm.cdf(d1) - params[1] * np_grad.exp(-params[4] * params[2]) * st_grad.norm.cdf(d2)
        else:
            d1 = (np_grad.log(self.params[0] / self.params[1]) + (self.params[4] + self.params[3] ** 2 / 2) * self.params[2]) / (np_grad.sqrt(self.params[2]) * self.params[3])
            d2 = d1 - np_grad.sqrt(self.params[2]) * self.params[3]
            return self.params[0] * st_grad.norm.cdf(d1) - self.params[1] * np_grad.exp(-self.params[4] * self.params[2]) * st_grad.norm.cdf(d2)
    
    def delta(self):
        d1 = (np.log(self.params[0] / self.params[1]) + (self.params[4] + self.params[3] ** 2 / 2) * self.params[2]) / (np.sqrt(self.params[2]) * self.params[3])
        return st.norm.cdf(d1)
    
    def aad_first_order_greeks(self):
        nabla_f = autograd.grad(self.price)
        greeks = nabla_f(self.params)
        # print(greeks)
        print(f"Delta of an option is equal to: {np.round(greeks[0], 4)}")
        print(f"Theta of an option is equal to: {-np.round(greeks[2] / 252, 4)}")
        print(f"Vega of an option is equal to: {np.round(greeks[3], 4)}")
        print(f"Rho of an option is equal to: {np.round(greeks[4], 4)}")
    
    def aad_second_order_greeks(self):
        hess_f = autograd.hessian(self.price)
        greeks = hess_f(self.params)
        # print(greeks)
        print(f"Gamma of an option is equal to: {np.round(greeks[0][0], 4)}")
        print(f"Vanna of an option is equal to: {np.round(greeks[0][3], 4)}")
        print(f"Volga of an option is equal to: {np.round(greeks[3][3], 4)}")
        print(f"Charn of an option is equal to: {-np.round(greeks[0][2], 4)}")

    def price_monte_carlo(self, params = []):

        if self.N == 0:
            raise ValueError("Enter positive number of paths")

        if len(params) != 0:
            spot, strike, T, vol, r = params[0], params[1], params[2], params[3], params[4]
        else:
            spot, strike, T, vol, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]

        arr = st.norm.rvs(size = (int(np_grad.round(T * 252)), self.N))
        paths = arr * (vol * np_grad.sqrt(T)) + (r - vol ** 2 / 2) * T
        paths = spot * np_grad.exp(paths)
        return (np_grad.fmax(paths[-1, :] - strike, 0) / np_grad.exp(r * T)).mean()
        
    def aad_mc_first_order_greeks(self):
        greeks = autograd.elementwise_grad(self.price_monte_carlo)(self.params)
        print(f"Delta of an option is equal to: {np.round(greeks[0], 4)}")
        print(f"Theta of an option is equal to: {-np.round(greeks[2] / 252, 4)}")
        print(f"Vega of an option is equal to: {np.round(greeks[3], 4)}")
        print(f"Rho of an option is equal to: {np.round(greeks[4], 4)}")
    
    def aad_mc_second_order_greeks(self):
        hess_f = autograd.hessian(self.price_monte_carlo)
        greeks = hess_f(self.params)
        return greeks

In [7]:

call_option(100.0, 110.0, 1.0, 0.2, 0.05, 1_000_00).aad_mc_second_order_greeks()

array([[ 0.00000000e+00,  0.00000000e+00,  3.95534561e-02,
         3.95534561e-01, -1.11022302e-16],
       [ 0.00000000e+00,  0.00000000e+00,  1.77589777e-02,
         0.00000000e+00,  3.55179555e-01],
       [ 3.95534561e-02,  1.77589777e-02, -2.03035719e+00,
         2.02266279e+01,  3.71162635e+01],
       [ 3.95534561e-01,  0.00000000e+00,  2.02266279e+01,
         4.49899884e+00, -7.10542736e-15],
       [ 0.00000000e+00,  3.55179555e-01,  3.71162635e+01,
         0.00000000e+00, -3.90697510e+01]])

In [10]:
call_option(100.0, 110.0, 1.0, 0.2, 0.05, 1_000_00).aad_first_order_greeks()

Delta of an option is equal to: 0.4496
Theta of an option is equal to: -0.0234
Vega of an option is equal to: 39.576
Rho of an option is equal to: 38.9247


In [30]:
# before we move to the full scale Monte-Carlo approach, lets start with something simple: evaluating Greeks without explicitly defining them

class lookback_call_option:

    def __init__(self, spot, strike, T, vol, r, N = 0):
        self.params =  np.array([spot, strike, T, vol, r])
        self.N = N

    def price_monte_carlo(self, params = []):

        if self.N == 0:
            raise ValueError("Enter positive number of paths")

        if len(params) != 0:
            spot, strike, T, vol, r = params[0], params[1], params[2], params[3], params[4]
        else:
            spot, strike, T, vol, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]

        arr = st.norm.rvs(size = (int(np_grad.round(T * 252)), self.N))
        paths = arr * (vol * np_grad.sqrt(T)) + (r - vol ** 2 / 2) * T
        paths = spot * np_grad.exp(paths)
        return (np_grad.fmax(np_grad.max(paths, axis = 0) - strike, 0) / np_grad.exp(r * T)).mean()
        
    def aad_mc_first_order_greeks(self):
        greeks = autograd.elementwise_grad(self.price_monte_carlo)(self.params)
        print(f"Delta of an option is equal to: {np.round(greeks[0], 4)}")
        print(f"Theta of an option is equal to: {-np.round(greeks[2] / 252, 4)}")
        print(f"Vega of an option is equal to: {np.round(greeks[3], 4)}")
        print(f"Rho of an option is equal to: {np.round(greeks[4], 4)}")
    
    def aad_mc_second_order_greeks(self):
        hess_f = autograd.hessian(self.price_monte_carlo)
        greeks = hess_f(self.params)
        return greeks

In [35]:
lookback_call_option(100, 110, 1, 0.2, 0.05, 100_000).aad_mc_first_order_greeks()

: 

In [9]:
import pandas as pd
x = lookback_call_option(100, 110, 1, 0.2, 0.05, 100_000_000).aad_mc_second_order_greeks()
df = pd.DataFrame(x)
df.rename(columns = {0: "Spot", 1: "Strike", 2: "T", 3: "Vol", 4: "Rate"}, index = {0: "Spot", 1: "Strike", 2: "T", 3: "Vol", 4: "Rate"}, inplace=True)


In [10]:
df

,Spot,Strike,T,Vol,Rate
Spot,0.000000,0.000000,0.458371,4.583713,-4.440892e-16
Strike,0.000000,0.000000,0.047561,0.000000,9.512294e-01
T,0.458371,0.047561,-12.472893,336.258234,9.940347e+01
Vol,4.583713,0.000000,336.258234,1070.725934,0.000000e+00
Rate,0.000000,0.951229,99.403475,0.000000,-1.046352e+02


In [53]:
# before we move to the full scale Monte-Carlo approach, lets start with something simple: evaluating Greeks without explicitly defining them

class asian_call_option:

    def __init__(self, spot, strike, T, vol, r, N = 0):
        self.params =  np.array([spot, strike, T, vol, r])
        self.N = N

    def price_monte_carlo(self, params = []):

        if self.N == 0:
            raise ValueError("Enter positive number of paths")

        if len(params) != 0:
            spot, strike, T, vol, r = params[0], params[1], params[2], params[3], params[4]
        else:
            spot, strike, T, vol, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]

        arr = st.norm.rvs(size = (int(np_grad.round(T * 252)), self.N))
        paths = arr * (vol * np_grad.sqrt(T)) + (r - vol ** 2 / 2) * T
        paths = spot * np_grad.exp(paths)
        return (np_grad.fmax(np_grad.mean(paths, axis = 0) - strike, 0) / np_grad.exp(r * T)).mean()
        
    def aad_mc_first_order_greeks(self):
        greeks = autograd.elementwise_grad(self.price_monte_carlo)(self.params)
        # print(f"Delta of an option is equal to: {np.round(greeks[0], 4)}")
        # print(f"Theta of an option is equal to: {-np.round(greeks[2] / 252, 4)}")
        # print(f"Vega of an option is equal to: {np.round(greeks[3], 4)}")
        # print(f"Rho of an option is equal to: {np.round(greeks[4], 4)}")
        return greeks
    
    def aad_mc_second_order_greeks(self):
        hess_f = autograd.hessian(self.price_monte_carlo)
        greeks = hess_f(self.params)
        print(f"Gamma of an option is equal to: {np.round(greeks[0][0], 4)}")
        print(f"Vanna of an option is equal to: {np.round(greeks[0][3], 4)}")
        print(f"Volga of an option is equal to: {np.round(greeks[3][3], 4)}")
        print(f"Charn of an option is equal to: {-np.round(greeks[0][2], 4)}")
        return greeks
    
    def delta_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([(1 + h / 100) * spot, strike, vol, T, r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * spot)
    
    def theta_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol, T * (1 - h / 100 ), r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * T) / 252
    
    def vega_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol * (1 + h / 100), T, r])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * vol)
    
    def rho_bump(self, h):
        pv1 = self.price_monte_carlo(self.params)
        spot, strike, vol, T, r = self.params[0], self.params[1], self.params[2], self.params[3], self.params[4]
        new_params = np.array([spot, strike, vol, T, r * (1 + h / 100)])
        pv2 = self.price_monte_carlo(new_params)
        return (pv2 - pv1) / (h / 100 * r)

In [72]:
lst = []
for i in range(10):
    lst.append(asian_call_option(100.0, 105.0, 1.0, 0.2, 0.05, 10000).delta_bump(2))

np.std(lst)

np.float64(0.004358011796621367)

In [69]:
lst = []
for i in range(10):
    lst.append(asian_call_option(100.0, 105.0, 1.0, 0.2, 0.05, 10000).aad_mc_first_order_greeks()[0])

np.std(lst)

np.float64(0.003894820227936032)